In [1]:
import os
import sys
module_path = os.path.abspath(os.path.join('../../..'))
if module_path not in sys.path:
    sys.path.append(module_path)

In [2]:
print(module_path)

/Users/revilooliver/Documents/quantum_computing/research/pauli_sandwitching/cut4mitigation


In [50]:
# author: Ji Liu email: ji.liu@anl.gov

import itertools, numpy
import circuit_cutter
import mlrecon_methods as ml
import numpy as np
import mthree

import qiskit
import qiskit.providers.aer.noise as noise
from qiskit import *
from qiskit import QuantumCircuit, QuantumRegister, ClassicalRegister, Aer, execute, transpile

from qiskit.transpiler import PassManager

from qiskit.converters import circuit_to_dag
from qiskit.visualization import dag_drawer, plot_histogram
from qiskit.compiler import assemble

from qiskit.tools.monitor import job_monitor, backend_monitor, backend_overview

from qiskit.providers.aer.noise import NoiseModel

from utils.utils import filter_results, dict_to_list, H_distance, total_counts, norm_dict
from vqe_utils import read_from_file, MeasureCircuit, find_commute_groups, evaluation
from mlrecon_methods import run_circuits, collect_fragment_circuits, organize_tomography_data_from_list

In [4]:
qiskit.__qiskit_version__

{'qiskit-terra': '0.22.2', 'qiskit-aer': '0.11.1', 'qiskit-ignis': '0.7.1', 'qiskit-ibmq-provider': '0.19.2', 'qiskit': '0.39.2', 'qiskit-nature': '0.3.0', 'qiskit-finance': None, 'qiskit-optimization': None, 'qiskit-machine-learning': None}

In [5]:
numpy.set_printoptions(linewidth = 200)

qubits = 5

simulation_backend = "qasm_simulator"

seed = 0
print_circuits = True

In [6]:
IBMQ.load_account()
provider = IBMQ.get_provider(hub='ibm-q-ornl', group='ornl', project='chm185')
provider.backends()

[<IBMQSimulator('ibmq_qasm_simulator') from IBMQ(hub='ibm-q-ornl', group='ornl', project='chm185')>,
 <IBMQBackend('ibmq_montreal') from IBMQ(hub='ibm-q-ornl', group='ornl', project='chm185')>,
 <IBMQBackend('ibmq_toronto') from IBMQ(hub='ibm-q-ornl', group='ornl', project='chm185')>,
 <IBMQBackend('ibmq_kolkata') from IBMQ(hub='ibm-q-ornl', group='ornl', project='chm185')>,
 <IBMQBackend('ibmq_mumbai') from IBMQ(hub='ibm-q-ornl', group='ornl', project='chm185')>,
 <IBMQBackend('ibmq_lima') from IBMQ(hub='ibm-q-ornl', group='ornl', project='chm185')>,
 <IBMQBackend('ibmq_belem') from IBMQ(hub='ibm-q-ornl', group='ornl', project='chm185')>,
 <IBMQBackend('ibmq_quito') from IBMQ(hub='ibm-q-ornl', group='ornl', project='chm185')>,
 <IBMQBackend('ibmq_guadalupe') from IBMQ(hub='ibm-q-ornl', group='ornl', project='chm185')>,
 <IBMQSimulator('simulator_statevector') from IBMQ(hub='ibm-q-ornl', group='ornl', project='chm185')>,
 <IBMQSimulator('simulator_mps') from IBMQ(hub='ibm-q-ornl', grou

In [7]:
device = provider.get_backend('ibmq_toronto')
device

<IBMQBackend('ibmq_toronto') from IBMQ(hub='ibm-q-ornl', group='ornl', project='chm185')>

In [8]:
device.configuration().to_dict()['max_shots']

32000

In [9]:
from qiskit import IBMQ, Aer
from qiskit.providers.aer.noise import NoiseModel
noise_model = NoiseModel.from_backend(device, gate_error = False, thermal_relaxation= False)
print(noise_model)

NoiseModel:
  Basis gates: ['cx', 'id', 'reset', 'rz', 'sx', 'x']
  Instructions with noise: ['measure']
  Qubits with noise: [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26]
  Specific qubit errors: [('measure', (0,)), ('measure', (1,)), ('measure', (2,)), ('measure', (3,)), ('measure', (4,)), ('measure', (5,)), ('measure', (6,)), ('measure', (7,)), ('measure', (8,)), ('measure', (9,)), ('measure', (10,)), ('measure', (11,)), ('measure', (12,)), ('measure', (13,)), ('measure', (14,)), ('measure', (15,)), ('measure', (16,)), ('measure', (17,)), ('measure', (18,)), ('measure', (19,)), ('measure', (20,)), ('measure', (21,)), ('measure', (22,)), ('measure', (23,)), ('measure', (24,)), ('measure', (25,)), ('measure', (26,))]


In [10]:
noisy_simulator = provider.get_backend('ibmq_qasm_simulator')

In [11]:
front_layer = QuantumCircuit.from_qasm_file("qasm/vqe_Lih_front.qasm")

In [12]:
mid_layer = QuantumCircuit.from_qasm_file("qasm/vqe_Lih_mid.qasm")

In [13]:
end_layer = QuantumCircuit.from_qasm_file("qasm/vqe_Lih_end.qasm")

In [14]:
orign_circuit = front_layer + mid_layer + end_layer

/var/folders/s4/3cvgkkn17hg628pzrzzjnhww0000gn/T/ipykernel_69726/743480212.py:1: DeprecationWarning: The QuantumCircuit.__add__() method is being deprecated.Use the compose() method which is more flexible w.r.t circuit register compatibility.
  orign_circuit = front_layer + mid_layer + end_layer


In [15]:
orign_circuit.draw()

┌───┐      ┌───────────────┐   ┌────────────────┐                 »
q_0: ──────┤ X ├──────┤ Ry(-0.003534) ├─■─┤ Ry(-0.0036834) ├─────────────────»
           ├───┤      ├───────────────┤ │ └────────────────┘┌───────────────┐»
q_1: ──────┤ X ├──────┤ Ry(-0.015869) ├─■─────────■─────────┤ Ry(-0.015751) ├»
     ┌─────┴───┴─────┐└───────────────┘           │         └───────────────┘»
q_2: ┤ Ry(0.0038243) ├────────────────────────────■─────────────────■────────»
     ├───────────────┤                                              │        »
q_3: ┤ Ry(-0.019851) ├──────────────────────────────────────────────■────────»
     └───────────────┘                                                       »
q_4: ────────────────────────────────────────────────────────────────────────»
                                                                             »
«                      
«q_0: ─────────────────
«                      
«q_1: ─────────────────
«     ┌───────────────┐
«q_2: ┤ Ry(0.0036205) ├
«     └┬──────────────┤
«q_3: ─┤ Ry(0.019873) ├
«      └──────────────┘
«q_4: ─────────────────
«

In [16]:
#checking circuit

In [17]:
def apply_checking_circuit(qc, ctrl_bits, ancilla_bits, side = None):
    if len(ctrl_bits) != len(ancilla_bits):
        print("Size mismatch")
        return None
    if side == 'front':
        for i in ancilla_bits:
            qc.h(i)
        for j,k in zip(ctrl_bits, ancilla_bits):
            qc.cz(j, k)
    elif side == 'end':
        for j,k in zip(ctrl_bits, ancilla_bits):
            qc.cz(j, k)
        for i in ancilla_bits:
            qc.h(i)
    else:
        print("Side undefined")

In [18]:
circuit = front_layer.copy()

In [19]:
apply_checking_circuit(circuit, [1], [4], side = 'front')
circuit += mid_layer
apply_checking_circuit(circuit, [1], [4], side = 'end')
circuit += end_layer

/var/folders/s4/3cvgkkn17hg628pzrzzjnhww0000gn/T/ipykernel_69726/3853186367.py:2: DeprecationWarning: The QuantumCircuit.__iadd__() method is being deprecated. Use the compose() (potentially with the inplace=True argument) and tensor() methods which are more flexible w.r.t circuit register compatibility.
  circuit += mid_layer
/var/folders/s4/3cvgkkn17hg628pzrzzjnhww0000gn/T/ipykernel_69726/3853186367.py:4: DeprecationWarning: The QuantumCircuit.__iadd__() method is being deprecated. Use the compose() (potentially with the inplace=True argument) and tensor() methods which are more flexible w.r.t circuit register compatibility.
  circuit += end_layer


In [20]:
circuit.draw()

┌───┐      ┌───────────────┐      ┌────────────────┐      »
q_0: ──────┤ X ├──────┤ Ry(-0.003534) ├────■─┤ Ry(-0.0036834) ├──────»
           ├───┤      ├───────────────┤    │ └────────────────┘      »
q_1: ──────┤ X ├──────┤ Ry(-0.015869) ├─■──■─────────■─────────────■─»
     ┌─────┴───┴─────┐└───────────────┘ │            │             │ »
q_2: ┤ Ry(0.0038243) ├──────────────────┼────────────■──────────■──┼─»
     ├───────────────┤                  │                       │  │ »
q_3: ┤ Ry(-0.019851) ├──────────────────┼───────────────────────■──┼─»
     └─────┬───┬─────┘                  │                          │ »
q_4: ──────┤ H ├────────────────────────■──────────────────────────■─»
           └───┘                                                     »
«                      
«q_0: ─────────────────
«     ┌───────────────┐
«q_1: ┤ Ry(-0.015751) ├
«     ├───────────────┤
«q_2: ┤ Ry(0.0036205) ├
«     └┬──────────────┤
«q_3: ─┤ Ry(0.019873) ├
«      └────┬───┬─────┘
«q_4: ──────┤ H ├──────
«           └───┘

In [21]:
pauli_list = read_from_file('LiH_hamiltonian.txt')
pauli_commute = find_commute_groups(pauli_list)

In [22]:
#generate individual measurment circuits that changes the basis
meas_qcs = []
for i in range(0, len(pauli_commute)):
    temp_qc = MeasureCircuit(pauli_commute[i], num_qubits = 4,num_qargs = 5)
    meas_qcs.append(temp_qc)
    print(temp_qc)

YYYY
     ┌──────────────┐
q_0: ┤ U(π/2,0,π/2) ├
     ├──────────────┤
q_1: ┤ U(π/2,0,π/2) ├
     ├──────────────┤
q_2: ┤ U(π/2,0,π/2) ├
     ├──────────────┤
q_3: ┤ U(π/2,0,π/2) ├
     └──────────────┘
q_4: ────────────────
                     
YYZZ
     ┌──────────────┐
q_0: ┤ U(π/2,0,π/2) ├
     ├──────────────┤
q_1: ┤ U(π/2,0,π/2) ├
     └──────────────┘
q_2: ────────────────
                     
q_3: ────────────────
                     
q_4: ────────────────
                     
ZZYY
                     
q_0: ────────────────
                     
q_1: ────────────────
     ┌──────────────┐
q_2: ┤ U(π/2,0,π/2) ├
     ├──────────────┤
q_3: ┤ U(π/2,0,π/2) ├
     └──────────────┘
q_4: ────────────────
                     
YYZX
     ┌──────────────┐
q_0: ┤ U(π/2,0,π/2) ├
     ├──────────────┤
q_1: ┤ U(π/2,0,π/2) ├
     └──────────────┘
q_2: ────────────────
      ┌────────────┐ 
q_3: ─┤ U(π/2,0,π) ├─
      └────────────┘ 
q_4: ────────────────
                     
ZXYY
       

In [23]:
circuit_list = []
for meas in meas_qcs:
    temp_circuit = circuit.copy()
    temp_circuit += meas
    circuit_list.append(temp_circuit)

/var/folders/s4/3cvgkkn17hg628pzrzzjnhww0000gn/T/ipykernel_69726/1717817760.py:4: DeprecationWarning: The QuantumCircuit.__iadd__() method is being deprecated. Use the compose() (potentially with the inplace=True argument) and tensor() methods which are more flexible w.r.t circuit register compatibility.
  temp_circuit += meas


In [24]:
circuit_list[-2].draw()

┌───┐      ┌───────────────┐      ┌────────────────┐┌────────────┐»
q_0: ──────┤ X ├──────┤ Ry(-0.003534) ├────■─┤ Ry(-0.0036834) ├┤ U(π/2,0,π) ├»
           ├───┤      ├───────────────┤    │ └────────────────┘└────────────┘»
q_1: ──────┤ X ├──────┤ Ry(-0.015869) ├─■──■─────────■───────────────────────»
     ┌─────┴───┴─────┐└───────────────┘ │            │                       »
q_2: ┤ Ry(0.0038243) ├──────────────────┼────────────■───────────────■───────»
     ├───────────────┤                  │                            │       »
q_3: ┤ Ry(-0.019851) ├──────────────────┼────────────────────────────■───────»
     └─────┬───┬─────┘                  │                                    »
q_4: ──────┤ H ├────────────────────────■────────────────────────────────────»
           └───┘                                                             »
«                                       
«q_0: ──────────────────────────────────
«        ┌───────────────┐┌────────────┐
«q_1: ─■─┤ Ry(-0.015751) ├┤ U(π/2,0,π) ├
«      │ ├───────────────┤├────────────┤
«q_2: ─┼─┤ Ry(0.0036205) ├┤ U(π/2,0,π) ├
«      │ └┬──────────────┤├────────────┤
«q_3: ─┼──┤ Ry(0.019873) ├┤ U(π/2,0,π) ├
«      │  └────┬───┬─────┘└────────────┘
«q_4: ─■───────┤ H ├────────────────────
«              └───┘

In [25]:
# cuts = []
# for i in range(0, 1):
#     if i == 0 or i == qubits - 1:
#         cut = [(circuit.qubits[i], 5),(circuit.qubits[i], 6)]
#     else:
#         cut = [(circuit.qubits[i], 5),(circuit.qubits[i], 4)]
#     cuts += cut

In [26]:
cuts = [(circuit.qubits[1], 3),(circuit.qubits[1], 5)]

In [27]:
fragments_list = []
wire_path_map_list = []
for temp_qc in circuit_list:
    print(temp_qc)
    fragments, wire_path_map = circuit_cutter.cut_circuit(temp_qc, cuts)
    fragments_list.append(fragments)
    wire_path_map_list.append(wire_path_map)

           ┌───┐      ┌───────────────┐      ┌────────────────┐┌──────────────┐»
q_0: ──────┤ X ├──────┤ Ry(-0.003534) ├────■─┤ Ry(-0.0036834) ├┤ U(π/2,0,π/2) ├»
           ├───┤      ├───────────────┤    │ └────────────────┘└──────────────┘»
q_1: ──────┤ X ├──────┤ Ry(-0.015869) ├─■──■─────────■─────────────────────────»
     ┌─────┴───┴─────┐└───────────────┘ │            │                         »
q_2: ┤ Ry(0.0038243) ├──────────────────┼────────────■────────────────■────────»
     ├───────────────┤                  │                             │        »
q_3: ┤ Ry(-0.019851) ├──────────────────┼─────────────────────────────■────────»
     └─────┬───┬─────┘                  │                                      »
q_4: ──────┤ H ├────────────────────────■──────────────────────────────────────»
           └───┘                                                               »
«                                         
«q_0: ────────────────────────────────────
«        ┌─────────────

           ┌───┐      ┌───────────────┐      ┌────────────────┐┌────────────┐»
q_0: ──────┤ X ├──────┤ Ry(-0.003534) ├────■─┤ Ry(-0.0036834) ├┤ U(π/2,0,π) ├»
           ├───┤      ├───────────────┤    │ └────────────────┘└────────────┘»
q_1: ──────┤ X ├──────┤ Ry(-0.015869) ├─■──■─────────■───────────────────────»
     ┌─────┴───┴─────┐└───────────────┘ │            │                       »
q_2: ┤ Ry(0.0038243) ├──────────────────┼────────────■───────────────■───────»
     ├───────────────┤                  │                            │       »
q_3: ┤ Ry(-0.019851) ├──────────────────┼────────────────────────────■───────»
     └─────┬───┬─────┘                  │                                    »
q_4: ──────┤ H ├────────────────────────■────────────────────────────────────»
           └───┘                                                             »
«                                         
«q_0: ────────────────────────────────────
«        ┌───────────────┐ ┌────────────┐ 
«q

           ┌───┐      ┌───────────────┐      ┌────────────────┐      »
q_0: ──────┤ X ├──────┤ Ry(-0.003534) ├────■─┤ Ry(-0.0036834) ├──────»
           ├───┤      ├───────────────┤    │ └────────────────┘      »
q_1: ──────┤ X ├──────┤ Ry(-0.015869) ├─■──■─────────■─────────────■─»
     ┌─────┴───┴─────┐└───────────────┘ │            │             │ »
q_2: ┤ Ry(0.0038243) ├──────────────────┼────────────■──────────■──┼─»
     ├───────────────┤                  │                       │  │ »
q_3: ┤ Ry(-0.019851) ├──────────────────┼───────────────────────■──┼─»
     └─────┬───┬─────┘                  │                          │ »
q_4: ──────┤ H ├────────────────────────■──────────────────────────■─»
           └───┘                                                     »
«                                    
«q_0: ───────────────────────────────
«     ┌───────────────┐┌────────────┐
«q_1: ┤ Ry(-0.015751) ├┤ U(π/2,0,π) ├
«     ├───────────────┤├────────────┤
«q_2: ┤ Ry(0.0036205) ├┤ U(π/

In [28]:
print(wire_path_map_list[0])

{Qubit(QuantumRegister(5, 'q'), 0): ((0, Qubit(QuantumRegister(4, 'q'), 1)),), Qubit(QuantumRegister(5, 'q'), 1): ((1, Qubit(QuantumRegister(3, 'q'), 2)), (0, Qubit(QuantumRegister(4, 'q'), 0)), (1, Qubit(QuantumRegister(3, 'q'), 1))), Qubit(QuantumRegister(5, 'q'), 2): ((0, Qubit(QuantumRegister(4, 'q'), 2)),), Qubit(QuantumRegister(5, 'q'), 3): ((0, Qubit(QuantumRegister(4, 'q'), 3)),), Qubit(QuantumRegister(5, 'q'), 4): ((1, Qubit(QuantumRegister(3, 'q'), 0)),)}


In [29]:
total_variants = ml.fragment_variants(wire_path_map_list[0])

In [30]:
total_variants

24

In [31]:
shots = device.configuration().to_dict()['max_shots'] * total_variants

In [32]:
fragments_list[0][0].draw()

»
q_0: ───────────────────────────────────■────────────────────■─────────────────»
           ┌───┐      ┌───────────────┐ │ ┌────────────────┐ │ ┌──────────────┐»
q_1: ──────┤ X ├──────┤ Ry(-0.003534) ├─■─┤ Ry(-0.0036834) ├─┼─┤ U(π/2,0,π/2) ├»
     ┌─────┴───┴─────┐└───────────────┘   └────────────────┘ │ └──────────────┘»
q_2: ┤ Ry(0.0038243) ├───────────────────────────────────────■────────■────────»
     ├───────────────┤                                                │        »
q_3: ┤ Ry(-0.019851) ├────────────────────────────────────────────────■────────»
     └───────────────┘                                                         »
«                                      
«q_0: ─────────────────────────────────
«                                      
«q_1: ─────────────────────────────────
«     ┌───────────────┐┌──────────────┐
«q_2: ┤ Ry(0.0036205) ├┤ U(π/2,0,π/2) ├
«     └┬──────────────┤├──────────────┤
«q_3: ─┤ Ry(0.019873) ├┤ U(π/2,0,π/2) ├
«      └──────────────┘└──────────────┘

In [33]:
fragment_cuts_list = []
for index in range(0, len(fragments_list)):
    for idx, fragment in enumerate(fragments_list[index]):
        print(f"fragment {idx}:")
        print(fragment)
        print()
    fragment_cuts = ml.fragment_cuts(wire_path_map_list[index])
    fragment_cuts_list.append(fragment_cuts)


fragment 0:
                                                                               »
q_0: ───────────────────────────────────■────────────────────■─────────────────»
           ┌───┐      ┌───────────────┐ │ ┌────────────────┐ │ ┌──────────────┐»
q_1: ──────┤ X ├──────┤ Ry(-0.003534) ├─■─┤ Ry(-0.0036834) ├─┼─┤ U(π/2,0,π/2) ├»
     ┌─────┴───┴─────┐└───────────────┘   └────────────────┘ │ └──────────────┘»
q_2: ┤ Ry(0.0038243) ├───────────────────────────────────────■────────■────────»
     ├───────────────┤                                                │        »
q_3: ┤ Ry(-0.019851) ├────────────────────────────────────────────────■────────»
     └───────────────┘                                                         »
«                                      
«q_0: ─────────────────────────────────
«                                      
«q_1: ─────────────────────────────────
«     ┌───────────────┐┌──────────────┐
«q_2: ┤ Ry(0.0036205) ├┤ U(π/2,0,π/2) ├
«     └┬───────────

     ┌───┐                             ┌───┐                    
q_0: ┤ H ├──────────────────■──■───────┤ H ├────────────────────
     └───┘                  │  │ ┌─────┴───┴─────┐┌────────────┐
q_1: ───────────────────────┼──■─┤ Ry(-0.015751) ├┤ U(π/2,0,π) ├
     ┌───┐┌───────────────┐ │    └───────────────┘└────────────┘
q_2: ┤ X ├┤ Ry(-0.015869) ├─■───────────────────────────────────
     └───┘└───────────────┘                                     

fragment 0:
                                                                  »
q_0: ───────────────────────────────────■────────────────────■────»
           ┌───┐      ┌───────────────┐ │ ┌────────────────┐ │    »
q_1: ──────┤ X ├──────┤ Ry(-0.003534) ├─■─┤ Ry(-0.0036834) ├─┼────»
     ┌─────┴───┴─────┐└───────────────┘   └────────────────┘ │    »
q_2: ┤ Ry(0.0038243) ├───────────────────────────────────────■──■─»
     ├───────────────┤                                          │ »
q_3: ┤ Ry(-0.019851) ├──────────────────────────────────

In [34]:
print(fragment_cuts_list[-1])

[{'prep': 1, 'meas': 1}, {'prep': 1, 'meas': 1}]


In [35]:
wire_path_map

{Qubit(QuantumRegister(5, 'q'), 0): ((0, Qubit(QuantumRegister(4, 'q'), 1)),),
 Qubit(QuantumRegister(5, 'q'), 1): ((1, Qubit(QuantumRegister(3, 'q'), 2)),
  (0, Qubit(QuantumRegister(4, 'q'), 0)),
  (1, Qubit(QuantumRegister(3, 'q'), 1))),
 Qubit(QuantumRegister(5, 'q'), 2): ((0, Qubit(QuantumRegister(4, 'q'), 2)),),
 Qubit(QuantumRegister(5, 'q'), 3): ((0, Qubit(QuantumRegister(4, 'q'), 3)),),
 Qubit(QuantumRegister(5, 'q'), 4): ((1, Qubit(QuantumRegister(3, 'q'), 0)),)}

In [36]:
num_fragments = len(fragments)
num_fragments

2

In [37]:
num_tomo_circuits = len(meas_qcs)
num_tomo_circuits

25

In [38]:
fragments[0].draw()

»
q_0: ───────────────────────────────────■────────────────────■────»
           ┌───┐      ┌───────────────┐ │ ┌────────────────┐ │    »
q_1: ──────┤ X ├──────┤ Ry(-0.003534) ├─■─┤ Ry(-0.0036834) ├─┼────»
     ┌─────┴───┴─────┐└───────────────┘   └────────────────┘ │    »
q_2: ┤ Ry(0.0038243) ├───────────────────────────────────────■──■─»
     ├───────────────┤                                          │ »
q_3: ┤ Ry(-0.019851) ├──────────────────────────────────────────■─»
     └───────────────┘                                            »
«                      
«q_0: ─────────────────
«                      
«q_1: ─────────────────
«     ┌───────────────┐
«q_2: ┤ Ry(0.0036205) ├
«     └┬──────────────┤
«q_3: ─┤ Ry(0.019873) ├
«      └──────────────┘

In [39]:
fragments[1].draw()

┌───┐                             ┌───┐      
q_0: ┤ H ├──────────────────■──■───────┤ H ├──────
     └───┘                  │  │ ┌─────┴───┴─────┐
q_1: ───────────────────────┼──■─┤ Ry(-0.015751) ├
     ┌───┐┌───────────────┐ │    └───────────────┘
q_2: ┤ X ├┤ Ry(-0.015869) ├─■─────────────────────
     └───┘└───────────────┘

In [40]:
hardware_index = 0

In [41]:
tomography_circuits = []
for idx in range(0, len(fragments_list)):
    fragments = fragments_list[idx]
    wire_path_map = wire_path_map_list[idx]
    temp_frag_circuits = [0] * len(fragments)
    for i in range(0, len(fragments)):
        if i == hardware_index:
            temp_frag_circuits[i] = collect_fragment_circuits(fragments[i], i, wire_path_map,
                                         shots = shots // total_variants,
                                         tomography_backend = simulation_backend, extra_qc = None)
        else:
            temp_frag_circuits[i] = collect_fragment_circuits(fragments[i], i, wire_path_map,
                                         shots = shots // total_variants,
                                         tomography_backend = simulation_backend)
    tomography_circuits.append(temp_frag_circuits)

In [42]:
#contains 25 basis
len(tomography_circuits)

25

In [43]:
#contains three fragments
len(tomography_circuits[0])

2

In [44]:
len(tomography_circuits[0][1])

1

In [45]:
#tomography circuits
len(tomography_circuits[0][1][0])

12

In [46]:
multi_exp_circuits = [[],[]]
for i in range(0, len(tomography_circuits)):
    #for each basis collect the three fragments:
    for j in range(0, num_fragments):
        fragment_circ = tomography_circuits[i][j][0]
        multi_exp_circuits[j] += fragment_circ

In [47]:
multi_exp_circuits[0][-200].draw()

┌─────────────────┐ ░                                      »
  q_0: ┤ U3(-1.9106,π,0) ├─░────────────────────────────────────■─»
       └─────────────────┘ ░       ┌───┐      ┌───────────────┐ │ »
  q_1: ────────────────────░───────┤ X ├──────┤ Ry(-0.003534) ├─■─»
                           ░ ┌─────┴───┴─────┐└───────────────┘   »
  q_2: ────────────────────░─┤ Ry(0.0038243) ├────────────────────»
                           ░ ├───────────────┤                    »
  q_3: ────────────────────░─┤ Ry(-0.019851) ├────────────────────»
                           ░ └───────────────┘                    »
c16_0: ═══════════════════════════════════════════════════════════»
                                                                  »
c16_1: ═══════════════════════════════════════════════════════════»
                                                                  »
c16_2: ═══════════════════════════════════════════════════════════»
                                                                  »
c16_3: ═══════════════════════════════════════════════════════════»
                                                                  »
«                                                                            ░ »
«  q_0: ───────────────────■─────────────────────────────────────────────────░─»
«       ┌────────────────┐ │ ┌──────────────┐                                ░ »
«  q_1: ┤ Ry(-0.0036834) ├─┼─┤ U(π/2,0,π/2) ├────────────────────────────────░─»
«       └────────────────┘ │ └──────────────┘┌───────────────┐┌────────────┐ ░ »
«  q_2: ───────────────────■────────■────────┤ Ry(0.0036205) ├┤ U(π/2,0,π) ├─░─»
«                                   │        └┬──────────────┤├────────────┤ ░ »
«  q_3: ────────────────────────────■─────────┤ Ry(0.019873) ├┤ U(π/2,0,π) ├─░─»
«                                             └──────────────┘└────────────┘ ░ »
«c16_0: ═══════════════════════════════════════════════════════════════════════»
«                                                                              »
«c16_1: ═══════════════════════════════════════════════════════════════════════»
«                                                                              »
«c16_2: ═══════════════════════════════════════════════════════════════════════»
«                                                                              »
«c16_3: ═══════════════════════════════════════════════════════════════════════»
«                                                                              »
«       ┌───┐      ┌─┐
«  q_0: ┤ H ├──────┤M├
«       └┬─┬┘      └╥┘
«  q_1: ─┤M├────────╫─
«        └╥┘ ┌─┐    ║ 
«  q_2: ──╫──┤M├────╫─
«         ║  └╥┘┌─┐ ║ 
«  q_3: ──╫───╫─┤M├─╫─
«         ║   ║ └╥┘ ║ 
«c16_0: ══╬═══╬══╬══╩═
«         ║   ║  ║    
«c16_1: ══╩═══╬══╬════
«             ║  ║    
«c16_2: ══════╩══╬════
«                ║    
«c16_3: ═════════╩════
«

In [48]:
frag_datas = [0] * num_fragments
for i in range(0, num_fragments):
    if i == hardware_index:
        frag_datas[i] = run_circuits(multi_exp_circuits[i], initial_layout = [14,16,11,8], 
                                     backend = device, shots = shots // total_variants, 
                                     optimization_level = 3, monitor_jobs = True)
    else:
        frag_datas[i] = run_circuits(multi_exp_circuits[i], initial_layout = None,
                                     backend = simulation_backend, shots = shots // total_variants,
                                     optimization_level = 3, monitor_jobs = True, noise_model = None)
       

job_id:  63777f1518d7482c7ef75525
Job Status: job is queued (14)    

IBMQJobApiError: '"HTTPSConnectionPool(host=\'api-qcon.quantum-computing.ibm.com\', port=443): Max retries exceeded with url: /api/Network/ibm-q-ornl/Groups/ornl/Projects/chm185/Jobs/63777f1518d7482c7ef75525/status/v/1 (Caused by NewConnectionError(\'<urllib3.connection.HTTPSConnection object at 0x7fd301d47c40>: Failed to establish a new connection: [Errno 8] nodename nor servname provided, or not known\'))"'

In [51]:
frag_datas[0] = [device.retrieve_job(job_id='63777f1518d7482c7ef75525').result()]

In [52]:
frag_datas[1] = run_circuits(multi_exp_circuits[1], initial_layout = None,
                             backend = simulation_backend, shots = shots // total_variants,
                             optimization_level = 3, monitor_jobs = True, noise_model = None)

job_id:  13cbc3a2-8084-4c3c-af18-4a98dc28b250
Job Status: job has successfully run


In [53]:
frag_datas[0][0].results[0].header.name

"(('S0', 'S0', 'S0', 'S0'), ('Z', 'Z', 'Z', 'Z'))"

In [54]:
num_tomo_circuits

25

In [55]:
test = [[], []]

In [56]:
results_list = [[], []]
counts_list = [[], []] 
for i in range(0, num_fragments): 
    for j in range(0, num_tomo_circuits):
        start_idx = j * 12
        end_idx = (j + 1) * 12
        results_list[i].append(frag_datas[i][0].results[start_idx : end_idx])
        counts_list[i].append(frag_datas[i][0].get_counts()[start_idx : end_idx])


In [57]:
frag_datas[0][0].get_counts()

[{'0000': 3680,
  '0001': 42,
  '0010': 3973,
  '0011': 35,
  '0100': 3571,
  '0101': 42,
  '0110': 3702,
  '0111': 41,
  '1000': 4205,
  '1001': 38,
  '1010': 4500,
  '1011': 47,
  '1100': 3871,
  '1101': 43,
  '1110': 4170,
  '1111': 40},
 {'0000': 2023,
  '0001': 1666,
  '0010': 2174,
  '0011': 1874,
  '0100': 2037,
  '0101': 1578,
  '0110': 2168,
  '0111': 1698,
  '1000': 2246,
  '1001': 1946,
  '1010': 2342,
  '1011': 2123,
  '1100': 2121,
  '1101': 1777,
  '1110': 2252,
  '1111': 1975},
 {'0000': 1876,
  '0001': 1846,
  '0010': 1983,
  '0011': 2025,
  '0100': 1842,
  '0101': 1783,
  '0110': 1919,
  '0111': 1893,
  '1000': 2112,
  '1001': 2032,
  '1010': 2382,
  '1011': 2163,
  '1100': 1977,
  '1101': 1931,
  '1110': 2068,
  '1111': 2168},
 {'0000': 1199,
  '0001': 2992,
  '0010': 1310,
  '0011': 2646,
  '0100': 998,
  '0101': 2747,
  '0110': 1045,
  '0111': 2315,
  '1000': 1165,
  '1001': 3209,
  '1010': 1299,
  '1011': 2831,
  '1100': 1153,
  '1101': 3071,
  '1110': 1250,
  '111

In [58]:
frag_datas[0][0].get_counts()

[{'0000': 3680,
  '0001': 42,
  '0010': 3973,
  '0011': 35,
  '0100': 3571,
  '0101': 42,
  '0110': 3702,
  '0111': 41,
  '1000': 4205,
  '1001': 38,
  '1010': 4500,
  '1011': 47,
  '1100': 3871,
  '1101': 43,
  '1110': 4170,
  '1111': 40},
 {'0000': 2023,
  '0001': 1666,
  '0010': 2174,
  '0011': 1874,
  '0100': 2037,
  '0101': 1578,
  '0110': 2168,
  '0111': 1698,
  '1000': 2246,
  '1001': 1946,
  '1010': 2342,
  '1011': 2123,
  '1100': 2121,
  '1101': 1777,
  '1110': 2252,
  '1111': 1975},
 {'0000': 1876,
  '0001': 1846,
  '0010': 1983,
  '0011': 2025,
  '0100': 1842,
  '0101': 1783,
  '0110': 1919,
  '0111': 1893,
  '1000': 2112,
  '1001': 2032,
  '1010': 2382,
  '1011': 2163,
  '1100': 1977,
  '1101': 1931,
  '1110': 2068,
  '1111': 2168},
 {'0000': 1199,
  '0001': 2992,
  '0010': 1310,
  '0011': 2646,
  '0100': 998,
  '0101': 2747,
  '0110': 1045,
  '0111': 2315,
  '1000': 1165,
  '1001': 3209,
  '1010': 1299,
  '1011': 2831,
  '1100': 1153,
  '1101': 3071,
  '1110': 1250,
  '111

In [59]:
from mlrecon_methods import organize_tomography_data_from_list
organized_data = []
for i in range(0, num_tomo_circuits):
    frag_data = []
    frag_targets = ml.identify_frag_targets(wire_path_map_list[i])
    for j in range(0, num_fragments):
        frag_data.append(organize_tomography_data_from_list(results_list[j][i], counts_list[j][i],
                                              frag_targets[j].get("prep"),
                                              frag_targets[j].get("meas"),
                                              prep_basis = "SIC", extra_qc = []))
    organized_data.append(frag_data)

(('S0', 'S0', 'S0', 'S0'), ('Z', 'Z', 'Z', 'Z')) ('S0', 'S0', 'S0', 'S0') ('Z', 'Z', 'Z', 'Z')
(('S0', 'S0', 'S0', 'S0'), ('Z', 'Z', 'Z', 'Z')) ('S0',) ('Z',) [0] [0]
(('S0', 'S0', 'S0', 'S0'), ('X', 'Z', 'Z', 'Z')) ('S0', 'S0', 'S0', 'S0') ('X', 'Z', 'Z', 'Z')
(('S0', 'S0', 'S0', 'S0'), ('X', 'Z', 'Z', 'Z')) ('S0',) ('X',) [0] [0]
(('S0', 'S0', 'S0', 'S0'), ('Y', 'Z', 'Z', 'Z')) ('S0', 'S0', 'S0', 'S0') ('Y', 'Z', 'Z', 'Z')
(('S0', 'S0', 'S0', 'S0'), ('Y', 'Z', 'Z', 'Z')) ('S0',) ('Y',) [0] [0]
(('S1', 'S0', 'S0', 'S0'), ('Z', 'Z', 'Z', 'Z')) ('S1', 'S0', 'S0', 'S0') ('Z', 'Z', 'Z', 'Z')
(('S1', 'S0', 'S0', 'S0'), ('Z', 'Z', 'Z', 'Z')) ('S1',) ('Z',) [0] [0]
(('S1', 'S0', 'S0', 'S0'), ('X', 'Z', 'Z', 'Z')) ('S1', 'S0', 'S0', 'S0') ('X', 'Z', 'Z', 'Z')
(('S1', 'S0', 'S0', 'S0'), ('X', 'Z', 'Z', 'Z')) ('S1',) ('X',) [0] [0]
(('S1', 'S0', 'S0', 'S0'), ('Y', 'Z', 'Z', 'Z')) ('S1', 'S0', 'S0', 'S0') ('Y', 'Z', 'Z', 'Z')
(('S1', 'S0', 'S0', 'S0'), ('Y', 'Z', 'Z', 'Z')) ('S1',) ('Y',) [0] [0

In [60]:
for final_bits, fixed_bit_data in frag_data[0].items():
    prep_meas_states, state_counts = zip(*fixed_bit_data.items())
    prep_labels, meas_labels = zip(*prep_meas_states)
    prep_qubit_num = len(prep_labels[0])
    meas_qubit_num = len(meas_labels[0])
    print(prep_qubit_num)

1
1
1
1
1
1
1
1


In [61]:
len(organized_data)

25

In [62]:
organized_data[2]

[{'000': {(('S0',), ('Zp',)): 271,
   (('S0',), ('Zm',)): 5,
   (('S0',), ('Xp',)): 130,
   (('S0',), ('Xm',)): 117,
   (('S0',), ('Yp',)): 142,
   (('S0',), ('Ym',)): 161,
   (('S1',), ('Zp',)): 78,
   (('S1',), ('Zm',)): 189,
   (('S1',), ('Xp',)): 36,
   (('S1',), ('Xm',)): 246,
   (('S1',), ('Yp',)): 150,
   (('S1',), ('Ym',)): 159,
   (('S2',), ('Zp',)): 88,
   (('S2',), ('Zm',)): 180,
   (('S2',), ('Xp',)): 208,
   (('S2',), ('Xm',)): 98,
   (('S2',), ('Yp',)): 215,
   (('S2',), ('Ym',)): 62,
   (('S3',), ('Zp',)): 112,
   (('S3',), ('Zm',)): 184,
   (('S3',), ('Xp',)): 194,
   (('S3',), ('Xm',)): 104,
   (('S3',), ('Yp',)): 59,
   (('S3',), ('Ym',)): 245},
  '001': {(('S0',), ('Zp',)): 7374,
   (('S0',), ('Zm',)): 71,
   (('S0',), ('Xp',)): 3884,
   (('S0',), ('Xm',)): 3521,
   (('S0',), ('Yp',)): 3708,
   (('S0',), ('Ym',)): 3935,
   (('S1',), ('Zp',)): 2193,
   (('S1',), ('Zm',)): 5492,
   (('S1',), ('Xp',)): 636,
   (('S1',), ('Xm',)): 7027,
   (('S1',), ('Yp',)): 3870,
   ((

In [63]:
organized_data[-1][-1]

{'11': {(('S0',), ('Zm',)): 3,
  (('S0',), ('Xm',)): 2,
  (('S0',), ('Xp',)): 1,
  (('S1',), ('Zm',)): 1,
  (('S1',), ('Zp',)): 1,
  (('S1',), ('Xm',)): 2,
  (('S1',), ('Ym',)): 2,
  (('S1',), ('Yp',)): 2,
  (('S2',), ('Zp',)): 1,
  (('S2',), ('Zm',)): 1,
  (('S2',), ('Xp',)): 2,
  (('S2',), ('Ym',)): 1,
  (('S2',), ('Yp',)): 1,
  (('S3',), ('Zp',)): 1,
  (('S3',), ('Xm',)): 1,
  (('S3',), ('Xp',)): 2,
  (('S3',), ('Yp',)): 1,
  (('S0',), ('Zp',)): 0,
  (('S0',), ('Yp',)): 0,
  (('S0',), ('Ym',)): 0,
  (('S1',), ('Xp',)): 0,
  (('S2',), ('Xm',)): 0,
  (('S3',), ('Zm',)): 0,
  (('S3',), ('Ym',)): 0},
 '00': {(('S0',), ('Zp',)): 3,
  (('S0',), ('Xm',)): 1,
  (('S0',), ('Ym',)): 2,
  (('S0',), ('Yp',)): 1,
  (('S1',), ('Xp',)): 3,
  (('S1',), ('Ym',)): 1,
  (('S2',), ('Zm',)): 2,
  (('S2',), ('Zp',)): 1,
  (('S2',), ('Xm',)): 2,
  (('S2',), ('Ym',)): 2,
  (('S3',), ('Zm',)): 1,
  (('S3',), ('Zp',)): 3,
  (('S3',), ('Xm',)): 2,
  (('S3',), ('Ym',)): 1,
  (('S3',), ('Yp',)): 2,
  (('S0',), 

In [64]:
recombined_lists = []
filtered_recombined_lists = []
norm_filtered_lists = []
for i in range(0, num_tomo_circuits):
    direct_models = ml.direct_fragment_model(organized_data[i])
    likely_models = ml.maximum_likelihood_model(direct_models)

    direct_recombined_dist = ml.recombine_fragment_models(direct_models, wire_path_map_list[i])
    likely_recombined_dist = ml.recombine_fragment_models(likely_models, wire_path_map_list[i])
    recombined_lists.append(likely_recombined_dist)
    filtered_recombined_lists.append(filter_results(likely_recombined_dist, [0]))
    norm_filtered_lists.append(norm_dict(filter_results(likely_recombined_dist, [0])))

{'000': {(('S0',), ('Zp',)): 3680, (('S0',), ('Zm',)): 42, (('S0',), ('Xp',)): 2023, (('S0',), ('Xm',)): 1666, (('S0',), ('Yp',)): 1876, (('S0',), ('Ym',)): 1846, (('S1',), ('Zp',)): 1199, (('S1',), ('Zm',)): 2992, (('S1',), ('Xp',)): 378, (('S1',), ('Xm',)): 3772, (('S1',), ('Yp',)): 2105, (('S1',), ('Ym',)): 2049, (('S2',), ('Zp',)): 1201, (('S2',), ('Zm',)): 2784, (('S2',), ('Xp',)): 2842, (('S2',), ('Xm',)): 1124, (('S2',), ('Yp',)): 3511, (('S2',), ('Ym',)): 462, (('S3',), ('Zp',)): 1355, (('S3',), ('Zm',)): 2713, (('S3',), ('Xp',)): 2768, (('S3',), ('Xm',)): 1125, (('S3',), ('Yp',)): 517, (('S3',), ('Ym',)): 3384}, '001': {(('S0',), ('Zp',)): 3973, (('S0',), ('Zm',)): 35, (('S0',), ('Xp',)): 2174, (('S0',), ('Xm',)): 1874, (('S0',), ('Yp',)): 1983, (('S0',), ('Ym',)): 2025, (('S1',), ('Zp',)): 1310, (('S1',), ('Zm',)): 2646, (('S1',), ('Xp',)): 268, (('S1',), ('Xm',)): 3627, (('S1',), ('Yp',)): 2011, (('S1',), ('Ym',)): 1860, (('S2',), ('Zp',)): 1314, (('S2',), ('Zm',)): 2513, ((

{'00': {(('S0',), ('Zp',)): 2, (('S0',), ('Xm',)): 1, (('S1',), ('Zm',)): 10693, (('S1',), ('Xm',)): 5313, (('S1',), ('Xp',)): 5289, (('S1',), ('Yp',)): 5370, (('S1',), ('Ym',)): 5333, (('S2',), ('Zm',)): 10693, (('S2',), ('Xp',)): 5245, (('S2',), ('Xm',)): 5434, (('S2',), ('Ym',)): 5248, (('S2',), ('Yp',)): 5381, (('S3',), ('Zm',)): 10765, (('S3',), ('Zp',)): 1, (('S3',), ('Xp',)): 5427, (('S3',), ('Xm',)): 5283, (('S3',), ('Yp',)): 5433, (('S3',), ('Ym',)): 5428, (('S0',), ('Zm',)): 0, (('S0',), ('Xp',)): 0, (('S0',), ('Yp',)): 0, (('S0',), ('Ym',)): 0, (('S1',), ('Zp',)): 0, (('S2',), ('Zp',)): 0}, '01': {(('S0',), ('Zm',)): 16085, (('S0',), ('Xm',)): 8070, (('S0',), ('Xp',)): 7932, (('S0',), ('Yp',)): 7940, (('S0',), ('Ym',)): 8021, (('S1',), ('Zp',)): 1, (('S1',), ('Zm',)): 5400, (('S1',), ('Xm',)): 2731, (('S1',), ('Xp',)): 2643, (('S1',), ('Ym',)): 2619, (('S1',), ('Yp',)): 2733, (('S2',), ('Zm',)): 5232, (('S2',), ('Xm',)): 2615, (('S2',), ('Xp',)): 2610, (('S2',), ('Yp',)): 26

{'000': {(('S0',), ('Zp',)): 305, (('S0',), ('Zm',)): 3, (('S0',), ('Xp',)): 149, (('S0',), ('Xm',)): 180, (('S0',), ('Yp',)): 165, (('S0',), ('Ym',)): 139, (('S1',), ('Zp',)): 77, (('S1',), ('Zm',)): 191, (('S1',), ('Xp',)): 62, (('S1',), ('Xm',)): 223, (('S1',), ('Yp',)): 154, (('S1',), ('Ym',)): 139, (('S2',), ('Zp',)): 112, (('S2',), ('Zm',)): 188, (('S2',), ('Xp',)): 209, (('S2',), ('Xm',)): 101, (('S2',), ('Yp',)): 223, (('S2',), ('Ym',)): 45, (('S3',), ('Zp',)): 110, (('S3',), ('Zm',)): 193, (('S3',), ('Xp',)): 185, (('S3',), ('Xm',)): 134, (('S3',), ('Yp',)): 59, (('S3',), ('Ym',)): 262}, '001': {(('S0',), ('Zp',)): 8235, (('S0',), ('Zm',)): 90, (('S0',), ('Xp',)): 3851, (('S0',), ('Xm',)): 4314, (('S0',), ('Yp',)): 4205, (('S0',), ('Ym',)): 4078, (('S1',), ('Zp',)): 2300, (('S1',), ('Zm',)): 5478, (('S1',), ('Xp',)): 658, (('S1',), ('Xm',)): 7059, (('S1',), ('Yp',)): 3840, (('S1',), ('Ym',)): 3857, (('S2',), ('Zp',)): 2699, (('S2',), ('Zm',)): 4996, (('S2',), ('Xp',)): 5508, (

{'11': {(('S0',), ('Zm',)): 1, (('S0',), ('Xm',)): 1, (('S0',), ('Xp',)): 1, (('S0',), ('Ym',)): 3, (('S1',), ('Zm',)): 2, (('S1',), ('Xm',)): 2, (('S1',), ('Yp',)): 2, (('S1',), ('Ym',)): 2, (('S2',), ('Zp',)): 1, (('S2',), ('Xp',)): 2, (('S2',), ('Ym',)): 1, (('S3',), ('Zp',)): 1, (('S3',), ('Xp',)): 1, (('S3',), ('Xm',)): 1, (('S3',), ('Ym',)): 1, (('S0',), ('Zp',)): 0, (('S0',), ('Yp',)): 0, (('S1',), ('Zp',)): 0, (('S1',), ('Xp',)): 0, (('S2',), ('Zm',)): 0, (('S2',), ('Xm',)): 0, (('S2',), ('Yp',)): 0, (('S3',), ('Zm',)): 0, (('S3',), ('Yp',)): 0}, '00': {(('S0',), ('Zp',)): 2, (('S0',), ('Xm',)): 1, (('S0',), ('Xp',)): 2, (('S0',), ('Ym',)): 2, (('S0',), ('Yp',)): 1, (('S1',), ('Zm',)): 2, (('S2',), ('Zm',)): 1, (('S2',), ('Zp',)): 1, (('S2',), ('Ym',)): 2, (('S3',), ('Zm',)): 1, (('S3',), ('Zp',)): 2, (('S3',), ('Xm',)): 2, (('S3',), ('Xp',)): 2, (('S3',), ('Yp',)): 2, (('S0',), ('Zm',)): 0, (('S1',), ('Zp',)): 0, (('S1',), ('Xp',)): 0, (('S1',), ('Xm',)): 0, (('S1',), ('Yp',))

In [65]:
# direct_models = ml.direct_fragment_model(frag_data)
# likely_models = ml.maximum_likelihood_model(direct_models)

# direct_recombined_dist = ml.recombine_fragment_models(direct_models, wire_path_map)
# likely_recombined_dist = ml.recombine_fragment_models(likely_models, wire_path_map)

In [66]:
direct_recombined_dist

{'10010': 1.2275216072799624e-06,
 '00000': 1.828807662074513e-07,
 '10000': 0.0008093183355258464,
 '00010': 0.034201600839634805,
 '10011': 4.581859899768237e-05,
 '00001': -2.7772303562024216e-05,
 '10001': 0.033619226901822154,
 '00011': 0.9122595579109294,
 '10110': 1.7853768864511157e-09,
 '00100': 3.667710685411923e-08,
 '10100': -7.952320500611285e-05,
 '00110': 0.00042128097730276455,
 '10111': -1.840450580526289e-07,
 '00101': 1.4566434133496327e-06,
 '10101': -0.00042762477384132905,
 '00111': 0.009258885307472441,
 '11010': -7.633446754492043e-09,
 '01000': 3.31783934151502e-08,
 '11000': -0.00011539444793481724,
 '01010': 0.00026034247410005966,
 '11011': 5.881363276956686e-07,
 '01001': -3.287345250207924e-07,
 '11001': 0.0012903804433019468,
 '01011': 0.008444400153996156,
 '11110': -1.8372572383266333e-09,
 '01100': 1.6921032018442312e-09,
 '11100': -1.033082471926627e-05,
 '01110': 5.254586514873622e-06,
 '11111': -4.418884275478342e-09,
 '01101': 2.0642505055432115e-0

In [67]:
likely_recombined_dist

{'10000': 0.0007945909296507632,
 '00010': 0.03350249916116387,
 '10001': 0.03659677904561295,
 '00011': 0.9112824854532131,
 '10101': 3.165012292140876e-05,
 '00111': 0.008599137092703891,
 '11001': 0.0008313818450950228,
 '01011': 0.008361476349638965}

In [68]:
filtered_recombined_lists

[{'0000': 0.06338864826975604,
  '0010': 0.06384023764380362,
  '0001': 0.057213962842027635,
  '0011': 0.05759497471419848,
  '0100': 0.058513270398584415,
  '0110': 0.05915896832509142,
  '0101': 0.053707275125720225,
  '0111': 0.05421779268267412,
  '1000': 0.0706290762413599,
  '1010': 0.07117335928790015,
  '1001': 0.06492452354804105,
  '1011': 0.06538475869576604,
  '1100': 0.0657896226513614,
  '1110': 0.0664789370850685,
  '1101': 0.060903159550072486,
  '1111': 0.06147276902247032},
 {'0010': 0.250975022737184,
  '0000': 0.24685224999691954,
  '0011': 0.2281763475218426,
  '0001': 0.22465650895643077,
  '0110': 0.0019240486232986154,
  '0100': 0.0018778391421915216,
  '0111': 0.0016799615478207785,
  '0101': 0.0016485854872087733,
  '1010': 0.0017956131080238322,
  '1000': 0.001763416765440709,
  '1011': 0.0019171324242717393,
  '1001': 0.0018807493036867488},
 {'0010': 0.006312417936635714,
  '0011': 0.23667258824623186,
  '0110': 0.005919638356748231,
  '0111': 0.2214997122

In [69]:
from utils.utils import filter_results

In [70]:
filter_direct_recombined = filter_results(direct_recombined_dist, [0])

In [71]:
filter_direct_recombined

{'0000': 1.828807662074513e-07,
 '0010': 0.034201600839634805,
 '0001': -2.7772303562024216e-05,
 '0011': 0.9122595579109294,
 '0100': 3.667710685411923e-08,
 '0110': 0.00042128097730276455,
 '0101': 1.4566434133496327e-06,
 '0111': 0.009258885307472441,
 '1000': 3.31783934151502e-08,
 '1010': 0.00026034247410005966,
 '1001': -3.287345250207924e-07,
 '1011': 0.008444400153996156,
 '1100': 1.6921032018442312e-09,
 '1110': 5.254586514873622e-06,
 '1101': 2.0642505055432115e-08,
 '1111': 5.2411129128898855e-05}

In [72]:

filter_likely_recombined = filter_results(likely_recombined_dist, [0])

In [73]:
filter_likely_recombined

{'0010': 0.03350249916116387,
 '0011': 0.9112824854532131,
 '0111': 0.008599137092703891,
 '1011': 0.008361476349638965}

In [74]:
norm_filter_dist = norm_dict(filter_likely_recombined)

In [75]:
likely_recombined_dist['00011']

0.9112824854532131

In [76]:
#normalized distribution after mitigation: 0.9189375->0.9286796265616319
norm_filter_dist['0011']

0.9475296661555079

In [77]:
norm_filter_dist

{'0010': 0.03483509488253257,
 '0011': 0.9475296661555079,
 '0111': 0.008941176450486596,
 '1011': 0.00869406251147285}

In [78]:
final_expect_val = 0
for i in range(0, len(pauli_commute)):
    group = pauli_commute[i]
    for Pauli_tuple in group:
        coeff = Pauli_tuple[1]
        final_expect_val += coeff * evaluation(norm_filtered_lists[i], shots = total_counts(norm_filtered_lists[i]), Pauli = Pauli_tuple[0])  

In [79]:
final_expect_val

(-1.0578289603343904+0j)

In [80]:
import json
with open('LiH_toronto_check1.json', 'w') as f:
    json.dump(norm_filtered_lists, f)